In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

def load_and_clean_data(filepath):
    """
    Loads the OnlineRetail.csv dataset and performs initial cleaning.

    Args:
        filepath (str): The path to the CSV file.

    Returns:
        pd.DataFrame: A cleaned DataFrame.
    """
    try:
        # Load the dataset
        df = pd.read_csv(filepath, encoding='latin1')
        print("Data loaded successfully.")
    except FileNotFoundError:
        print(f"Error: The file at {filepath} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    # --- Step 1: Data Cleaning (Simplified for pre-cleaned file) ---
    print("\n--- Performing minimal data cleaning on the provided file ---")

    # Convert 'InvoiceDate' to datetime objects for time-series analysis
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

    # Drop any rows with remaining missing values, just to be safe
    initial_rows = len(df)
    df.dropna(inplace=True)
    print(f"Removed {initial_rows - len(df)} rows with missing values.")

    # Ensure StockCode is treated as a string
    df['StockCode'] = df['StockCode'].astype(str)

    print(f"Total rows after cleaning: {len(df)}")

    return df

def perform_data_aggregation_and_mining(df):
    """
    Performs data aggregation for time-series analysis and market basket analysis.

    Args:
        df (pd.DataFrame): The cleaned DataFrame.

    Returns:
        tuple: A tuple containing the time-series DataFrame and the association rules DataFrame.
    """
    # --- Step 2: Feature Engineering & Aggregation for Time-Series ---
    print("\n--- Aggregating data for time-series forecasting ---")

    # Group by date and product to get daily sales
    daily_sales = df.groupby(['InvoiceDate', 'StockCode'])['Quantity'].sum().reset_index()

    # Pivot the table to have products as columns and dates as rows
    time_series_df = daily_sales.pivot_table(
        index='InvoiceDate', columns='StockCode', values='Quantity'
    ).fillna(0)

    print("Time-series DataFrame created successfully.")
    print(time_series_df.head())

    # --- Step 3: Association Rule Mining (Market Basket Analysis) ---
    print("\n--- Performing Market Basket Analysis using Apriori ---")

    # To avoid memory issues, we will focus the market basket analysis on a single country
    basket_df = df[df['Country'] == 'United Kingdom']

    # Create a new DataFrame for market basket analysis using a more stable method.
    basket_sets = basket_df.pivot_table(index='InvoiceNo', columns='Description', values='Quantity', aggfunc='sum').fillna(0).applymap(lambda x: 1 if x > 0 else 0)

    # Apply the Apriori algorithm to find frequent itemsets
    frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)

    # Generate association rules with confidence and lift metrics
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

    print("\nAssociation rules generated successfully.")
    print("--- Top 5 Association Rules (by lift) ---")
    print(rules.sort_values('lift', ascending=False).head())

    return time_series_df, rules


if __name__ == '__main__':
    file_path = 'Cleaned_OnlineRetail.csv'
    cleaned_df = load_and_clean_data(file_path)

    if cleaned_df is not None:
        time_series_df, rules_df = perform_data_aggregation_and_mining(cleaned_df)

        # Now you can use time_series_df for Phase 2 (AI Model Training)
        # and rules_df for business insights and the agent's logic.


Data loaded successfully.

--- Performing minimal data cleaning on the provided file ---
Removed 132220 rows with missing values.
Total rows after cleaning: 397884

--- Aggregating data for time-series forecasting ---
Time-series DataFrame created successfully.
StockCode            10002  10080  10120  10123C  10124A  10124G  10125  \
InvoiceDate                                                               
2010-12-01 08:26:00    0.0    0.0    0.0     0.0     0.0     0.0    0.0   
2010-12-01 08:28:00    0.0    0.0    0.0     0.0     0.0     0.0    0.0   
2010-12-01 08:34:00    0.0    0.0    0.0     0.0     0.0     0.0    0.0   
2010-12-01 08:35:00    0.0    0.0    0.0     0.0     0.0     0.0    0.0   
2010-12-01 08:45:00   48.0    0.0    0.0     0.0     0.0     0.0    0.0   

StockCode            10133  10135  11001  ...  90214V  90214W  90214Y  90214Z  \
InvoiceDate                               ...                                   
2010-12-01 08:26:00    0.0    0.0    0.0  ...     

In [9]:
!pip install pandas numpy tensorflow scikit-learn statsmodels mlxtend keras

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
import warnings

warnings.filterwarnings('ignore')

def train_model(time_series_df):
    """
    Trains a hybrid ARIMA-LSTM model for demand forecasting.

    Args:
        time_series_df (pd.DataFrame): The time-series data from data_preparation.py.
    """
    # --- Step 1: Select a single product for a proof of concept ---
    # We will pick '85123A' as it is a top-selling product
    if '85123A' not in time_series_df.columns:
        print("Product '85123A' not found in DataFrame. Please select a valid product code.")
        return None, None, None, None

    product_data = time_series_df['85123A'].values.reshape(-1, 1)

    # Check for empty or all-zero data
    if np.sum(product_data) == 0:
        print("Selected product has no sales data. Cannot train model.")
        return None, None, None, None

    # --- Step 2: Data Preprocessing for LSTM ---
    # Scale the data to be between 0 and 1, which LSTMs prefer
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(product_data)

    # Create sequences for the LSTM model
    def create_sequences(data, seq_length):
        xs, ys = [], []
        for i in range(len(data) - seq_length):
            x = data[i:(i + seq_length)]
            y = data[i + seq_length]
            xs.append(x)
            ys.append(y)
        return np.array(xs), np.array(ys)

    # Use a sequence length of 7 (for a week)
    sequence_length = 7
    if len(scaled_data) < sequence_length + 1:
        print("Not enough data to create sequences. Please use a longer time series.")
        return None, None, None, None

    X, y = create_sequences(scaled_data, sequence_length)

    # Split data into training and testing sets
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # --- Step 3: Implement the Hybrid ARIMA-LSTM Model ---
    # Part A: ARIMA model to capture linear trends
    try:
        arima_model = ARIMA(product_data[:train_size].flatten(), order=(1, 1, 1))
        arima_model_fit = arima_model.fit()
        arima_residuals = arima_model_fit.resid
    except Exception as e:
        print(f"ARIMA model failed to fit: {e}. Skipping ARIMA step.")
        # If ARIMA fails, we can still try to train the LSTM on the raw data
        arima_residuals = product_data[:train_size].flatten()

    # Part B: LSTM model to capture non-linear patterns from residuals
    # Scale the residuals
    residual_scaler = MinMaxScaler(feature_range=(0, 1))

    # Ensure there are enough residuals to train the LSTM model
    if len(arima_residuals) < sequence_length + 1:
        print("Not enough residuals to create LSTM sequences.")
        return None, None, None, None

    scaled_residuals = residual_scaler.fit_transform(arima_residuals.reshape(-1, 1))

    # Create sequences for the LSTM from the scaled residuals
    X_res, y_res = create_sequences(scaled_residuals, sequence_length)

    if len(X_res) < 1:
        print("Not enough data after creating sequences from residuals.")
        return None, None, None, None

    X_res_train = X_res
    y_res_train = y_res

    # Reshape for LSTM
    X_res_train = X_res_train.reshape((X_res_train.shape[0], X_res_train.shape[1], 1))

    # Build the LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, activation='relu', input_shape=(X_res_train.shape[1], X_res_train.shape[2])))
    lstm_model.add(Dense(1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the LSTM model
    print("\nTraining LSTM model...")
    lstm_model.fit(X_res_train, y_res_train, epochs=20, verbose=1)

    print("\nModel training complete.")

    # --- Step 4: Model Evaluation (for a single step prediction) ---
    print("\n--- Evaluating Model Performance ---")

    # Make a prediction on the test data
    # Predict residuals using the LSTM model
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    lstm_predictions_scaled = lstm_model.predict(X_test_reshaped)

    # Inverse transform the scaled predictions to original scale
    lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)

    # Make a prediction with ARIMA on the test data
    try:
        arima_forecast = arima_model_fit.forecast(steps=len(X_test)).to_numpy().reshape(-1, 1)
    except:
        arima_forecast = np.zeros_like(lstm_predictions) # Fallback if ARIMA failed

    # Combine the predictions
    final_predictions = arima_forecast[:len(lstm_predictions)] + lstm_predictions

    # Inverse transform the original test data
    actual_values = scaler.inverse_transform(y_test[:len(final_predictions)])

    # Calculate Mean Absolute Error (MAE)
    mae = np.mean(np.abs(final_predictions - actual_values))
    print(f"Mean Absolute Error (MAE) on test data: {mae:.2f}")

    return arima_model_fit, lstm_model, scaler, residual_scaler

if __name__ == '__main__':
    # You must run data_preparation.py first to get this DataFrame
    # For now, we will create a dummy DataFrame to allow the code to be run independently
    # In your final project, you would import the actual DataFrame

    # Create a dummy DataFrame to represent daily sales for a single product
    dummy_dates = pd.date_range(start='2010-12-01', end='2011-12-09', freq='D')
    dummy_data = np.random.randint(0, 100, size=(len(dummy_dates), 1))

    dummy_df = pd.DataFrame(data=dummy_data, index=dummy_dates, columns=['85123A'])

    arima_model, lstm_model, main_scaler, res_scaler = train_model(dummy_df)


Training LSTM model...
Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 326ms/step - loss: 0.2851
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1923 
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1224 
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0799 
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0766 
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0742 
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0785 
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0748 
Epoch 9/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0755 
Epoch 10/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0731 
Epoch 11/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0743 
Epoch 12/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0759 
Epoch 13/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0792 
Epoch 14/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0771 
Epoch 15/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0680 
Epoch 16/

In [14]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

# We need to recreate the models and scalers from the previous step for this file to be runnable on its own.
# In a real project, these trained models would be saved and loaded from files.

def create_dummy_models_and_data():
    """
    Creates dummy data and trains simplified dummy models to make the
    agent_creation.py file runnable on its own.
    In a real project, this would be replaced with loading your trained models.
    """
    print("--- Creating dummy models and data for agent simulation ---")

    # Dummy data
    dummy_dates = pd.date_range(start='2010-12-01', end='2011-12-09', freq='D')
    dummy_data = np.random.randint(0, 100, size=(len(dummy_dates), 1))

    # Dummy ARIMA model
    arima_model = ARIMA(dummy_data[:200].flatten(), order=(1, 1, 1))
    arima_model_fit = arima_model.fit()

    # Dummy LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, activation='relu', input_shape=(7, 1)))
    lstm_model.add(Dense(1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')

    # Dummy scalers
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(dummy_data)

    return arima_model_fit, lstm_model, scaler

def forecast_demand(arima_model_fit, lstm_model, scaler, historical_data, forecast_horizon=7):
    """
    Generates a demand forecast for the next 'forecast_horizon' days.

    Args:
        arima_model_fit: The fitted ARIMA model object.
        lstm_model: The trained LSTM model object.
        scaler: The MinMaxScaler used for the data.
        historical_data (np.array): The most recent historical data for the model's sequence length.
        forecast_horizon (int): The number of days to forecast into the future.

    Returns:
        np.array: An array of forecasted demand for the next 'forecast_horizon' days.
    """
    print(f"\n--- Forecasting demand for the next {forecast_horizon} days ---")

    forecasts = []
    current_data = historical_data.copy()

    for _ in range(forecast_horizon):
        # 1. Forecast with ARIMA
        # Use the most recent data to forecast the next step
        arima_forecast = arima_model_fit.forecast(steps=1)

        # 2. Scale the current data for LSTM input
        current_data_scaled = scaler.transform(current_data.reshape(-1, 1))

        # 3. Predict the non-linear component (residual) with LSTM
        lstm_input = current_data_scaled.reshape(1, current_data_scaled.shape[0], 1)
        lstm_residual_scaled = lstm_model.predict(lstm_input, verbose=0)
        lstm_residual = scaler.inverse_transform(lstm_residual_scaled)[0, 0]

        # 4. Combine the forecasts
        combined_forecast = arima_forecast + lstm_residual

        forecasts.append(combined_forecast)

        # 5. Update the historical data for the next prediction step
        current_data = np.append(current_data[1:], combined_forecast)


    return np.array(forecasts).flatten()

def calculate_restock_order(forecast, current_stock, lead_time=3, safety_stock_multiplier=1.2):
    """
    Calculates the optimal restocking order quantity based on forecast and current inventory.

    Args:
        forecast (np.array): Array of forecasted demand.
        current_stock (int): The current number of items in stock.
        lead_time (int): Time in days to receive a new order.
        safety_stock_multiplier (float): Multiplier for safety stock.

    Returns:
        int: The recommended restocking order quantity.
    """
    print("\n--- Calculating restock order ---")

    # Calculate demand over the lead time period
    demand_during_lead_time = np.sum(forecast[:lead_time])

    # Calculate safety stock
    average_daily_demand = np.mean(forecast)
    safety_stock = int(average_daily_demand * safety_stock_multiplier)

    # Calculate optimal restock point
    restock_point = demand_during_lead_time + safety_stock

    # Determine the order quantity
    if current_stock < restock_point:
        order_quantity = restock_point - current_stock
        print(f"Current Stock: {current_stock}, Restock Point: {restock_point}")
        print(f"Demand during lead time: {demand_during_lead_time}, Safety Stock: {safety_stock}")
        print(f"Order recommended: {int(order_quantity)} units.")
        return int(order_quantity)
    else:
        print(f"Current Stock ({current_stock}) is above the restock point ({restock_point}). No order needed.")
        return 0

if __name__ == '__main__':
    # --- Step 1: Initialize the models and data for a single product ---
    # In a real application, you would load your trained models and the latest data

    # Get a dummy model and scaler
    arima_model, lstm_model, scaler = create_dummy_models_and_data()

    # We need recent historical data to make a new forecast
    historical_data = np.random.randint(0, 100, size=(7,))

    # --- Step 2: The Agent's Automation Loop ---
    # This loop represents the daily or weekly automation

    # --- Part A: Forecast future demand ---
    forecasted_demand = forecast_demand(
        arima_model_fit=arima_model,
        lstm_model=lstm_model,
        scaler=scaler,
        historical_data=historical_data
    )

    print("\nForecasted demand for the next 7 days:")
    print(np.round(forecasted_demand, 2))

    # --- Part B: Calculate the restocking order ---
    # Simulate a current stock level from a sensor or database
    current_inventory = 50

    restock_order = calculate_restock_order(
        forecast=forecasted_demand,
        current_stock=current_inventory
    )

    print(f"\nFinal recommended order quantity: {restock_order} units.")

--- Creating dummy models and data for agent simulation ---

--- Forecasting demand for the next 7 days ---



Forecasted demand for the next 7 days:
[51.36 51.43 51.19 51.19 50.9  50.97 51.21]

--- Calculating restock order ---
Current Stock: 50, Restock Point: 214.98068842957593
Demand during lead time: 153.98068842957593, Safety Stock: 61
Order recommended: 164 units.

Final recommended order quantity: 164 units.
